In [1]:
# no tracking server
# local backend and artifact store

import mlflow
import pandas as pd
import os
import numpy as np
import torch
from fastai.vision.all import *
import random
from torchvision.models import resnet50, ResNet50_Weights

ModuleNotFoundError: No module named 'torch._C'

In [ ]:
def seed_everything(seed=0):
    """ ensure reproducability by defining random seed"""
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything()

In [ ]:
os.environ['TORCH_HOME'] = 'models\\resnet' #setting the environment variable
path = os.path.join(os.getcwd(), 'data')  # path to downloaded dataset

In [2]:
print(f"local tracking uri {mlflow.get_tracking_uri()}")
mlflow.list_experiments()

local tracking uri file:///C:/Users/g7kaemj/Documents/mlops_zoomcamp_homework/06_project/mlruns


[<Experiment: artifact_location='file:///C:/Users/g7kaemj/Documents/mlops_zoomcamp_homework/06_project/mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>]

In [ ]:
def preprocess_data(df, metadata):
    """ add label column and change path to fname column"""
    df_lego = pd.merge(df, metadata[['minifigure_name']], on='class_id')
    df_lego['labels'] = df['minifigure_name'].apply(lambda x: x.lower())
    df_lego['fname'] = df['path']
    return df_lego

In [ ]:
mlflow.set_experiment("project_resnet50_first")

with mlflow.start_run():
    df = pd.read_csv(path/'index.csv', index_col=1)
    lego_metadata = pd.read_csv(path/'metadata.csv', index_col=0)
    df_lego = preprocess_data(df, lego_metadata)
    data = ImageDataLoaders.from_df(df_lego, path, valid_pct=0.10,
                                   item_tfms=Resize(412),
                                   bs=10, num_workers=0, label_col="labels")
    resnet = resnet50(weights=ResNet50_Weights.DEFAULT)
    learn = vision_learner(data, resnet50, metrics=[error_rate, accuracy], model_dir = Path(os.path.join(os.getcwd(),
                                                                                                   "models",
                                                                                                      "resnet")),
                       path =Path(""
                                                                                                                    "."))
    params = {"lr1": 1e-3, "lr2": 1e-1}
    mlflow.log_params(params)
    learn.fit_one_cycle(100,slice(params["lr1"],params["lr2"]),cbs=[EarlyStoppingCallback(patience=2),
                                                                    ShowGraphCallback(),
                                                SaveModelCallback(fname='model_best'),ReduceLROnPlateau()])